In [1]:
%env MINERL_DATA_ROOT=/Volumes/CORSAIR/data

env: MINERL_DATA_ROOT=/Volumes/CORSAIR/data


In [2]:
import minerl  # NOTE: we need gym>=0.13.1,<0.20
import gym
from train import get_agent, get_dynamics_environment, FMC, get_data_handler
from fgz.training.fgz_trainer import FGZTrainer
import torch

/Users/dyllan/miniconda3/envs/minerl/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.empty_cache()  # fix memory leaks

In [4]:
minerl_env = gym.make('MineRLBasaltFindCave-v0')
agent = get_agent()
dynamics_env = get_dynamics_environment(minerl_env)

Loading model /Volumes/CORSAIR/data/VPT-models/foundation-model-1x.model
with weights /Volumes/CORSAIR/data/VPT-models/foundation-model-1x.weights


In [5]:
# dynamics_env.batched_action_space_sample()

In [6]:
# dummy_initial_state = torch.ones(4096, dtype=float)
# dynamics_env.set_all_states(dummy_initial_state)

In [7]:
data_handler = get_data_handler(agent)
str(data_handler.loaders[0].trajectories[0])

'T(Player347-d527194e85c7-20220706-234429)'

In [8]:
# t = data_handler.sample_single_trajectory()
# for window in t:
#     print(len(window))
#     for frame, state_embedding, action in window:
#         print(state_embedding.shape)
#         break
#     break

In [9]:
fmc = FMC(dynamics_env, freeze_best=True)
trainer = FGZTrainer(minerl_env, agent, fmc, data_handler, unroll_steps=8)

In [10]:
trainer.train_trajectory(use_tqdm=True)

Sliding T Window:   0%|          | 0/3649 [00:00<?, ?it/s]/Users/dyllan/miniconda3/envs/minerl/lib/python3.7/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
Sliding T Window:   0%|          | 0/3649 [00:00<?, ?it/s]


In [11]:
# fmc.simulate(4, use_tqdm=True)

In [12]:
# fmc.tree.render() ;

In [13]:
# for state, action in fmc.tree.best_path:
#     print(state.reward, state.observation.var(), action)
#     print()

In [14]:
# trainer.train_trajectory()